In [29]:
class Account:
    month = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    season = ["ns", "ns", "ns", "ns", "s", "s", "s", "s", "s", "s", "ns", "ns"]
    user_type = {"la" : "la","lv" : "lv","hv" : "hv"}

    def __init__(self, id, name):
        self.id = id
        self.name = name
        self.contract = {"s" : self.init_values(),"ns" : self.init_values()}
        self.unitcprice = {"s" : self.init_values(),"ns" : self.init_values()}
        self.demand = {}
        self.baseprice = {}
        self.penaltyprice = {}
        self.penaltyprice_stage_1 = {}
        self.penaltyprice_stage_2 = {}
        self.totalprice = {}

    
        
    #Method for input the demand
    #demand_r: regular demand
    #demand_ns = non-summer demand
    #demand_sat = Saturday demand
    #demant_op = off-peak demand


    def init_values(self):
        return {"r" : 0, "hp" :0, "ns": 0, "shp" : 0,"op" : 0}
    
    def set_values(self, dict, values):
        dict = {"r" : values[0], "hp" :values[1], "ns": values[2], "shp" : values[3],"op" : values[4]}
        return dict

    def set_contract(self, season = "s", values = [0,0,0,0,0]): 
        self.contract[season] = self.set_values(self.contract[season], values)

    def set_demand(self, month_index, values = [0,0,0,0,0]): 
        self.demand[month_index] = self.set_values(self.demand, values)
        
    def set_unitcontractprice(self, season = "s", values = [0,0,0,0,0]): 
        self.unitcprice[season] = self.set_values(self.unitcprice[season], values)

    def get_contract(self, month_index):
        s = self.season[month_index]
        contract_month = self.contract[s]
        return contract_month

    def get_demand(self, month_index):
        demand_month = self.demand[month_index]
        return demand_month
    
    def get_unitcontractprice(self, month_index):
        s = self.season[month_index]
        unitcprice_month = self.unitcprice[s]
        return unitcprice_month
        
    def caculate_exceed(self, contract, demand):
        cacuEx = lambda x, y : x-y if x-y > 0 else 0 
        exceed = {
            "r" : cacuEx(demand["r"],contract["r"]),
            "hp" :cacuEx(demand["hp"],contract["r"]+contract["hp"]),
            "ns" : cacuEx(demand["ns"],contract["r"]+contract["ns"]),
            "shp" : cacuEx(demand["shp"],contract["r"]+contract["ns"]+contract["hp"]+contract["shp"]),
            "op" : cacuEx(demand["op"],contract["r"]+contract["ns"]+contract["hp"]+contract["shp"]+contract["op"])
            }
        return exceed
    
    
    def calculate_baseprice(self, contract, unitcprice):
        common = contract["shp"]+contract["op"]-(contract["r"]+contract["ns"])*0.5
        return contract["r"]*unitcprice["r"]+contract["ns"]*unitcprice["ns"]+contract["hp"]*unitcprice["hp"]+contract["shp"]*common
    

    def caculate_penaltyprice(self, contract, exceed, unitcprice):
        cacuStage_1 = lambda x, y: max(x - y*0.1, 0)

        hp = max(exceed["hp"]-exceed["r"], 0)
        shp = max(exceed["shp"]-max(exceed["r"], exceed["hp"]), 0)
        op = max(exceed["op"]-max(exceed["r"],exceed["hp"], exceed["shp"]), 0)

        penalty = {
            "r" : {
                "stage_1" : cacuStage_1(exceed["r"], contract["r"]),
                "stage_2" : exceed["r"] - cacuStage_1(exceed["r"], contract["r"])
                
            },
            "hp" : {
                "stage_1" : cacuStage_1(hp, contract["hp"]),
                "stage_2" : hp - cacuStage_1(hp, contract["hp"])
            },
            "ns" : {
                "stage_1" : cacuStage_1(exceed["ns"], contract["ns"]),
                "stage_2" : exceed["ns"] - cacuStage_1(exceed["ns"], contract["ns"])
            },
            "shp" : {
                "stage_1" : cacuStage_1(shp, contract["shp"]),
                "stage_2" : shp - cacuStage_1(shp, contract["shp"])
            },
            "op" : {
                "stage_1" : cacuStage_1(op, contract["op"]),
                "stage_2" : op - cacuStage_1(op, contract["op"])
            }
        }
  
        penaltyprice = {
             "r" : {
                "stage_1" : penalty["r"]["stage_1"] * unitcprice["r"] * 2,
                "stage_2" : penalty["r"]["stage_2"] * unitcprice["r"] * 3
            },
            "hp" : {
                "stage_1" : penalty["hp"]["stage_1"] * unitcprice["hp"] * 2,
                "stage_2" : penalty["hp"]["stage_2"] * unitcprice["hp"] * 3
            },
            "ns" : {
                "stage_1" : penalty["ns"]["stage_1"] * unitcprice["ns"] * 2,
                "stage_2" : penalty["ns"]["stage_2"] * unitcprice["ns"] * 3
            },
            "shp" : {
                "stage_1" : penalty["shp"]["stage_1"] * unitcprice["shp"] * 2,
                "stage_2" : penalty["shp"]["stage_2"] * unitcprice["shp"] * 3
            },
            "op" : {
                "stage_1" : penalty["op"]["stage_1"] * unitcprice["op"] * 2,
                "stage_2" : penalty["op"]["stage_2"] * unitcprice["op"] * 3
            }

        } 
        return penaltyprice

    
    def caculate_total(self):
        for month_index in range(12):
            contract = self.get_contract(month_index)
            demand = self.get_demand(month_index)
            exceed = self.caculate_exceed(contract, demand)
            unitcprice = self.get_unitcontractprice(month_index)
            self.baseprice[month_index] = self.calculate_baseprice(contract,unitcprice)
            self.penaltyprice[month_index] = self.caculate_penaltyprice(contract, exceed, unitcprice)
            self.penaltyprice_stage_1[month_index] = sum([self.penaltyprice[month_index][key]["stage_1"] for key in self.penaltyprice[month_index].keys()])
            self.penaltyprice_stage_2[month_index] = sum([self.penaltyprice[month_index][key]["stage_2"] for key in self.penaltyprice[month_index].keys()])

            self.totalprice[month_index] = self.calculate_baseprice(contract,unitcprice) + self.penaltyprice_stage_1[month_index] + self.penaltyprice_stage_2[month_index]

In [30]:
user1 = Account(1000,'C1')

In [31]:
user1.set_unitcontractprice("s", [223.6,0,166.9,44.7,44.7])
user1.set_unitcontractprice("ns", [166.9,0,166.9,33.3,33.3])
user1.unitcprice

{'s': {'r': 223.6, 'hp': 0, 'ns': 166.9, 'shp': 44.7, 'op': 44.7},
 'ns': {'r': 166.9, 'hp': 0, 'ns': 166.9, 'shp': 33.3, 'op': 33.3}}

In [32]:
user1.set_contract("s", [300,0,0,0,0])
user1.set_contract("ns", [300,0,0,0,0])
user1.contract

{'s': {'r': 300, 'hp': 0, 'ns': 0, 'shp': 0, 'op': 0},
 'ns': {'r': 300, 'hp': 0, 'ns': 0, 'shp': 0, 'op': 0}}

In [33]:
demand = [[68,0,0,62,36],
          [66,0,0,42,37],
          [177,0,0,154,36],
          [278,0,0,179,150],
          [276,0,0,164,39],
          [309,0,0,201,40],
          [357,0,0,215,48],
          [329,0,0,205,174],
          [342,0,0,312,54],
          [326,0,0,234,47],
          [240,0,0,173,92],
          [157,0,0,153,36]]

for month_index in range(12):
   user1.set_demand(month_index, demand[month_index])
   print(user1.month[month_index], user1.demand[month_index])

Jan {'r': 68, 'hp': 0, 'ns': 0, 'shp': 62, 'op': 36}
Feb {'r': 66, 'hp': 0, 'ns': 0, 'shp': 42, 'op': 37}
Mar {'r': 177, 'hp': 0, 'ns': 0, 'shp': 154, 'op': 36}
Apr {'r': 278, 'hp': 0, 'ns': 0, 'shp': 179, 'op': 150}
May {'r': 276, 'hp': 0, 'ns': 0, 'shp': 164, 'op': 39}
Jun {'r': 309, 'hp': 0, 'ns': 0, 'shp': 201, 'op': 40}
Jul {'r': 357, 'hp': 0, 'ns': 0, 'shp': 215, 'op': 48}
Aug {'r': 329, 'hp': 0, 'ns': 0, 'shp': 205, 'op': 174}
Sep {'r': 342, 'hp': 0, 'ns': 0, 'shp': 312, 'op': 54}
Oct {'r': 326, 'hp': 0, 'ns': 0, 'shp': 234, 'op': 47}
Nov {'r': 240, 'hp': 0, 'ns': 0, 'shp': 173, 'op': 92}
Dec {'r': 157, 'hp': 0, 'ns': 0, 'shp': 153, 'op': 36}


In [54]:
#return the total price of each month
user1.caculate_total()



table = {"month" : user1.month, "baseprice" : user1.baseprice, "penaltyprice_stage_1" : user1.penaltyprice_stage_1, "penaltyprice_stage_2" : user1.penaltyprice_stage_2, "totalprice" : user1.totalprice}
import pandas as pd
#print the baseprice, penaltyprice_stage_1, penaltyprice_stage_2, totalprice of each month in a table
df = pd.DataFrame(table, columns = ["month", "baseprice", "penaltyprice_stage_1", "penaltyprice_stage_2", "totalprice"], index = [0,1,2,3,4,5,6,7,8,9,10,11])
df

,month,baseprice,penaltyprice_stage_1,penaltyprice_stage_2,totalprice
0,Jan,50070.0,0.0,0.0,50070.0
1,Feb,50070.0,0.0,0.0,50070.0
2,Mar,50070.0,0.0,0.0,50070.0
3,Apr,50070.0,0.0,0.0,50070.0
4,May,67080.0,0.0,0.0,67080.0
5,Jun,67080.0,0.0,6037.2,73117.2
6,Jul,67080.0,12074.4,20124.0,99278.4
7,Aug,67080.0,0.0,19453.2,86533.2
8,Sep,67080.0,5366.4,20124.0,92570.4
9,Oct,67080.0,0.0,17440.8,84520.8


In [ ]:
# create a function to choose the best contract for each month
def chooseBestContract(user, month_index):
    #caculate the total price of each contract
    totalprice = {}
    for season in ["s", "ns"]:
        contract = user.getContract(month_index)
        demand = user.getDemand(month_index)
        exceed = user.cacuExceedcontract(contract, demand)
        unitcprice = user.getUnitcontractprice(month_index)
        penaltyprice = user.getPenaltyprice(contract, exceed, unitcprice)
        baseprice = user.getBaseprice(contract,unitcprice)
        totalprice[season] = baseprice + penaltyprice[season]["stage_1"] + penaltyprice[season]["stage_2"]
    #choose the best contract
    if totalprice["s"] < totalprice["ns"]:
        return "s"
    else:
        return "ns"